In [ ]:
import pandas as pd
import panel as pn
import holoviews as hv
import numpy as np
import panel as pn

In [58]:
def iplot(data_frame):
    # (1) Check if DataFrame has at least 3 columns
    if len(data_frame.columns) < 3:
        raise ValueError("The DataFrame should have at least 3 columns. Use a different function for this data.")

    # Protect the first dimension by reshaping -- reason is because pandas treats
    #     the first dimension in a dataframe in a special way and its name is harder to extract.
    if len(data_frame) != 1:
        raise ValueError("Data frame must have only one row. Please reshape your data accordingly.")

    # Reshape DataFrame to multi-dimensional array for easier indexing.
    #     Refer to pandas documentation if this is confusing; persist and it'll make sense.
    df_shape = [len(data_frame.index)] + [len(level) for level in data_frame.columns.levels]
    df_shape = df_shape[1:]
    data = data_frame.values.reshape(df_shape)

    # Grab index names for auto-axis updates
    index_names = data_frame.columns.names

    # Define plotting functions
    def plot_1D(*indices):
        # print(f'indices={indices}')
        # idx = [slice(None) if i < 2 else indices[i-2] for i in range(len(df_shape))]
        # print(f'idx={idx}', flush=True)
        return hv.Curve(data[tuple(indices)])

    def plot_2D(*indices, kdims):
        return hv.Image(data[tuple(indices)], kdims=kdims)

    # Create widgets
    dim_selector = pn.widgets.RadioBoxGroup(name='Dimension', options=['1D', '2D'], inline=True)
    
    sliders = [pn.widgets.IntSlider(name=index_names[i], start=0, end=df_shape[i]-1) for i in range(len(index_names))]

    special_dim_0 = pn.widgets.IntInput(name='Special Dimension 0', value=0, step=1)
    special_dim_1 = pn.widgets.IntInput(name='Special Dimension 1', value=1, step=1)

    # Bind slider names to updated info
    @pn.depends(special_dim_0.param.value, special_dim_1.param.value)
    def update_slider_names(special_dim_0_value, special_dim_1_value):
        for dim_idx, slider in enumerate(sliders):
            if dim_idx == special_dim_0_value:
                slider.name = f'{index_names[dim_idx]} (IGNORE SLIDER -- Plot Axis 0)'
            elif dim_idx == special_dim_1_value:
                slider.name = f'{index_names[dim_idx]} (IGNORE SLIDER in 2D MODE -- Plot Axis 1)'
            else:
                slider.name = f'{index_names[dim_idx]}'

    # Bind plot updates to the widgets
    @pn.depends(dim_selector.param.value, *sliders, special_dim_0.param.value, special_dim_1.param.value)
    def reactive_plot(dim, *indices):
        dim = 1 if dim == '1D' else 2
        special_dims = indices[-2:]
        indices = indices[:-2]
        plot_method = plot_1D if dim == 1 else lambda *x : plot_2D(*x, kdims=[index_names[special_dims[0]], index_names[special_dims[1]]])
        if( special_dims[0] == special_dims[1] and dim == 2 ):
            print(f'Plotted dimensions are the same, dimension = {special_dims[0]}, no update to plot taken', end='\r')
            return
        idx = [slice(None) if i in special_dims[:dim] else indices[i] for i in range(len(indices))]
        return plot_method(*idx)

    # Create layout and return
    layout = pn.Row(
        pn.Column(dim_selector, *sliders, special_dim_0, special_dim_1, update_slider_names),
        reactive_plot
    )
    return layout

In [60]:
# Necessary setup from panel -- read docs if this confuses you.
pn.extension()

# Seed for reproducibility
np.random.seed(0)

# Setup the shape and populate random data
data_shape = (1, 50, 50, 30, 20) 
random_data = np.random.randn(*data_shape) 

# Create a DataFrame with a multi-level column index
columns = pd.MultiIndex.from_product([range(dim_size) for dim_size in data_shape[1:]],
                                     names=[f'CUSTOM_NAME {i}' for i in range(1,len(data_shape))])
df = pd.DataFrame(random_data.reshape(data_shape[0], -1), columns=columns)

# Create interactive plot layout
layout = iplot(df)

# Display the layout
layout.servable()

Row
    [0] Column
        [0] RadioBoxGroup(inline=True, name='Dimension', options=['1D', '2D'], value='1D')
        [1] IntSlider(end=49, name='CUSTOM_NAME 1...)
        [2] IntSlider(end=49, name='CUSTOM_NAME 2...)
        [3] IntSlider(end=29, name='CUSTOM_NAME 3')
        [4] IntSlider(end=19, name='CUSTOM_NAME 4')
        [5] IntInput(name='Special Dimension 0')
        [6] IntInput(name='Special Dimension 1', value=1)
        [7] ParamFunction(function, _pane=Str, defer_load=False)
    [1] ParamFunction(function, _pane=HoloViews, defer_load=False)